<a href="https://colab.research.google.com/github/abayashoknair/leafdisease-detection/blob/main/Streamlitapp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip install -U ipykernel

In [3]:
!pip install -q streamlit

     |████████████████████████████████| 10.1 MB 16.8 MB/s 
     |████████████████████████████████| 4.3 MB 41.3 MB/s 
     |████████████████████████████████| 76 kB 5.5 MB/s 
     |████████████████████████████████| 111 kB 62.4 MB/s 
     |████████████████████████████████| 181 kB 52.1 MB/s 
     |████████████████████████████████| 164 kB 56.5 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 


In [ ]:
!pip install streamlit
!pip install pyngrok

In [ ]:
!mkdir -p /drive/ngrok-ssh
%cd /drive/ngrok-ssh
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip -O ngrok-stable-linux-amd64.zip
!unzip -u ngrok-stable-linux-amd64.zip
!cp /drive/ngrok-ssh/ngrok /ngrok
!chmod +x /ngrok

In [6]:
#@title !/ngrok authtoken XXXXXXX { display-mode: "form" }
!/ngrok authtoken xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [7]:
%%writefile img_classification.py
import keras
from PIL import Image, ImageOps
import numpy as np
import cv2

def teachable_machine_classification(img, weights_file):
    # Load the model
    model = keras.models.load_model(weights_file)

    # Create the array of the right shape to feed into the keras model
    data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
    image = img
    #image sizing
    size = (224, 224)
    image = ImageOps.fit(image, size, Image.ANTIALIAS)

    #turn the image into a numpy array
    #image = cv2.imread(image)
    #image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    image_array = np.asarray(image)
    image_array = cv2.merge([image_array,image_array,image_array])
    # Normalize the image
    normalized_image_array = (image_array.astype(np.float32) / 255)

    # Load the image into the array
    data[0] = normalized_image_array

    # run the inference
    prediction = model.predict(data)
    #return np.argmax(prediction)
    return prediction.round() # return position of the highest probability

Writing img_classification.py


In [8]:
%%writefile app.py
import streamlit as st
from PIL import Image
from img_classification import teachable_machine_classification

st.sidebar.title("About")
st.sidebar.info("This is ML based leaf disease detection web app, where you can upload leaf images and predict whether the leaf is healthy or has rust or powdery diseases. This model is built with CNN.")
st.sidebar.title("detect any disease in leaves ")
uploaded_file = st.sidebar.file_uploader("Choose a leaf image ...", type="jpeg")

st.title("Leaf-disease-detection using CNN")
st.header("Predicting disease of leaf from leaf images")
st.text("Upload a leaf Image for image classification as healthy or diseased")


if uploaded_file is not None:
  if st.button('detect disease'):
    showpred = 1
    image = Image.open(uploaded_file)
    st.image(image, caption='Uploaded leaf image.', use_column_width='True')
    st.write("")
    st.write("Classifying...")
    label = teachable_machine_classification(image, '/content/sample_data')
    if label == 0:
        st.write("the leaf is healthy")
    else:
        st.write("The leaf has rusty disease")
    else:st.write("the ;eaf has powdery deisease")


Writing app.py


In [9]:
get_ipython().system_raw('./ngrok http 8501 &')

In [11]:
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    'import sys, json; print("Execute the next cell and the go to the following URL: " +json.load(sys.stdin)["tunnels"][0]["public_url"])'

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/usr/lib/python3.7/json/__init__.py", line 296, in load
    parse_constant=parse_constant, object_pairs_hook=object_pairs_hook, **kw)
  File "/usr/lib/python3.7/json/__init__.py", line 348, in loads
    return _default_decoder.decode(s)
  File "/usr/lib/python3.7/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/usr/lib/python3.7/json/decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)


In [ ]:
!streamlit run app.py

2022-04-21 05:39:43.732 INFO    numexpr.utils: NumExpr defaulting to 2 threads.

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://34.91.233.168:8501

